## 處理遺漏值

In [3]:
import pandas as pd
from io import StringIO

csv_data = '''A,B,C,D
1.0,2.0,3.0,4.0
5.0,6.0,,8.0
10.0,11.0,12.0,'''

df = pd.read_csv(StringIO(csv_data))
df

A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN

In [4]:
# 檢查是否為空值
df.isnull()

A      B      C      D
0  False  False  False  False
1  False  False   True  False
2  False  False  False   True

In [6]:
# 欄位的空值數量
df.isnull().sum()

A    0
B    0
C    1
D    1
dtype: int64

### 刪除遺漏值  

In [10]:
# 將後兩筆 1 2 刪除了
df.dropna()

A    B    C    D
0  1.0  2.0  3.0  4.0

In [15]:
df.dropna(axis=0)

A    B    C    D
0  1.0  2.0  3.0  4.0

In [17]:
df.dropna(axis=1)

A     B
0   1.0   2.0
1   5.0   6.0
2  10.0  11.0

In [18]:
# only drop rows where all columns are NaN
df.dropna(how='all')


A     B     C    D
0   1.0   2.0   3.0  4.0
1   5.0   6.0   NaN  8.0
2  10.0  11.0  12.0  NaN

In [19]:
# drop rows that have not at least 4 non-NaN values
df.dropna(thresh=4)

A    B    C    D
0  1.0  2.0  3.0  4.0

In [20]:
# only drop rows where NaN appear in specific columns (here: 'C')
df.dropna(subset=['C'])

A     B     C    D
0   1.0   2.0   3.0  4.0
2  10.0  11.0  12.0  NaN

### Impot (補值) 

In [23]:
from sklearn.preprocessing import Imputer
# 補 行 的平均 ; strategy='mean'or'median'or'most_frequen'
imr = Imputer(missing_values='NaN', strategy='mean', axis=0)
imr = imr.fit(df)
imputed_data = imr.transform(df.values)
imputed_data

array([[  1. ,   2. ,   3. ,   4. ],
       [  5. ,   6. ,   7.5,   8. ],
       [ 10. ,  11. ,  12. ,   6. ]])

In [14]:

df.values

array([[  1.,   2.,   3.,   4.],
       [  5.,   6.,  nan,   8.],
       [ 10.,  11.,  12.,  nan]])

### 處理分類數據


In [15]:
import pandas as pd

df = pd.DataFrame([['green', 'M', 10.1, 'class1'],
                   ['red', 'L', 13.5, 'class2'],
                   ['blue', 'XL', 15.3, 'class1']])

df.columns = ['color', 'size', 'price', 'classlabel']
df

color size  price classlabel
0  green    M   10.1     class1
1    red    L   13.5     class2
2   blue   XL   15.3     class1

# 有序特徵 ex:size , XL＞L＞M  
# 名目特徵 ex:color

### 對應有序特徵

In [16]:
size_mapping = {'XL': 3,
                'L': 2,
                'M': 1}

df['size'] = df['size'].map(size_mapping)
df

color  size  price classlabel
0  green     1   10.1     class1
1    red     2   13.5     class2
2   blue     3   15.3     class1

In [18]:
# 反向對應
inv_size_mapping = {v: k for k, v in size_mapping.items()}
df['size'].map(inv_size_mapping)


0     M
1     L
2    XL
Name: size, dtype: object

### 將類別標籤　轉成　整數值

In [32]:
df['classlabel']

0    class1
1    class2
2    class1
Name: classlabel, dtype: object

In [33]:
np.unique(df['classlabel'])

array(['class1', 'class2'], dtype=object)

In [31]:
# scikit-learn 大多會自動轉
import numpy as np

class_mapping = {label: idx for idx, label in enumerate(np.unique(df['classlabel']))}
class_mapping


{'class1': 0, 'class2': 1}

In [29]:
#內建函數 (function) enumerate() ，回傳以參數 (parameter) iterable 與連續整數配對的 enumerate 物件， start 為整數的起始值，預設為 0 
d = ['Spring', 'Summer', 'Fall', 'Winter']
for i, j in enumerate(d, 1):
    print(i, j)


1 Spring
2 Summer
3 Fall
4 Winter


In [34]:
df['classlabel'] = df['classlabel'].map(class_mapping)
df

color  size  price  classlabel
0  green     1   10.1           0
1    red     2   13.5           1
2   blue     3   15.3           0

In [35]:

inv_class_mapping = {v: k for k, v in class_mapping.items()}
df['classlabel'] = df['classlabel'].map(inv_class_mapping)
df

color  size  price classlabel
0  green     1   10.1     class1
1    red     2   13.5     class2
2   blue     3   15.3     class1

In [37]:
# 其實直接用 scikit LabelEncoder 比較快

from sklearn.preprocessing import LabelEncoder

class_le = LabelEncoder()
y = class_le.fit_transform(df['classlabel'].values)
y

array([0, 1, 0], dtype=int64)

In [20]:
class_le.inverse_transform(y)


array(['class1', 'class2', 'class1'], dtype=object)

### 對名目特徵做　one-hot encoding

In [39]:
X = df[['color', 'size', 'price']].values
X

array([['green', 1, 10.1],
       ['red', 2, 13.5],
       ['blue', 3, 15.3]], dtype=object)

In [40]:

color_le = LabelEncoder()
X[:, 0] = color_le.fit_transform(X[:, 0])
X

array([[1, 1, 10.1],
       [2, 2, 13.5],
       [0, 3, 15.3]], dtype=object)

In [41]:

from sklearn.preprocessing import OneHotEncoder

ohe = OneHotEncoder(categorical_features=[0])
ohe.fit_transform(X).toarray()

array([[  0. ,   1. ,   0. ,   1. ,  10.1],
       [  0. ,   0. ,   1. ,   2. ,  13.5],
       [  1. ,   0. ,   0. ,   3. ,  15.3]])

In [42]:
pd.get_dummies(df[['price', 'color', 'size']])


price  size  color_blue  color_green  color_red
0   10.1     1           0            1          0
1   13.5     2           0            0          1
2   15.3     3           1            0          0

# 將數據集分成訓練集和測試集

In [43]:
df_wine = pd.read_csv('https://archive.ics.uci.edu/'
                      'ml/machine-learning-databases/wine/wine.data',
                      header=None)

df_wine.columns = ['Class label', 'Alcohol', 'Malic acid', 'Ash',
                   'Alcalinity of ash', 'Magnesium', 'Total phenols',
                   'Flavanoids', 'Nonflavanoid phenols', 'Proanthocyanins',
                   'Color intensity', 'Hue', 'OD280/OD315 of diluted wines',
                   'Proline']

print('Class labels', np.unique(df_wine['Class label']))
df_wine.head()

Class labels [1 2 3]


Class label  Alcohol  Malic acid   Ash  Alcalinity of ash  Magnesium  \
0            1    14.23        1.71  2.43               15.6        127   
1            1    13.20        1.78  2.14               11.2        100   
2            1    13.16        2.36  2.67               18.6        101   
3            1    14.37        1.95  2.50               16.8        113   
4            1    13.24        2.59  2.87               21.0        118   

   Total phenols  Flavanoids  Nonflavanoid phenols  Proanthocyanins  \
0           2.80        3.06                  0.28             2.29   
1           2.65        2.76                  0.26             1.28   
2           2.80        3.24                  0.30             2.81   
3           3.85        3.49                  0.24             2.18   
4           2.80        2.69                  0.39             1.82   

   Color intensity   Hue  OD280/OD315 of diluted wines  Proline  
0             5.64  1.04                          3.92     1065  
1             4.38  1.05                          3.40     1050  
2             5.68  1.03                          3.17     1185  
3             7.80  0.86                          3.45     1480  
4             4.32  1.04                          2.93      735

In [44]:
from sklearn.model_selection import train_test_split

X, y = df_wine.iloc[:, 1:].values, df_wine.iloc[:, 0].values

X_train, X_test, y_train, y_test = \
    train_test_split(X, y, test_size=0.3, random_state=0)

## 縮放特徵 


In [47]:
# 最大最小縮放　normalized
from sklearn.preprocessing import MinMaxScaler
mms = MinMaxScaler()
X_train_norm = mms.fit_transform(X_train)
X_test_norm = mms.transform(X_test)

X_test_norm

array([[ 0.72849462,  0.16386555,  0.47849462,  0.29896907,  0.52173913,
         0.53956835,  0.74311927,  0.13461538,  0.37974684,  0.4364852 ,
         0.32478632,  0.70695971,  0.60566802],
       [ 0.47311828,  0.37394958,  0.60215054,  0.58762887,  0.45652174,
         0.13669065,  0.27217125,  0.19230769,  0.26582278,  0.90926457,
        -0.05128205,  0.07326007,  0.13603239],
       [ 0.36021505,  0.05042017,  0.43010753,  0.43298969,  0.18478261,
         0.86330935,  0.80428135,  0.09615385,  0.45886076,  0.30276982,
         0.58119658,  0.58608059,  0.08744939],
       [ 0.68010753,  0.17647059,  0.59139785,  0.51030928,  0.5       ,
         0.66906475,  0.70642202,  0.11538462,  0.64240506,  0.47468959,
         0.37606838,  0.64468864,  0.65425101],
       [ 0.54301075,  1.03151261,  0.41397849,  0.56185567,  0.17391304,
         0.54676259,  0.66666667,  0.30769231,  0.50316456,  0.1260745 ,
         0.16239316,  0.67032967,  0.05506073],
       [ 0.14247312,  0.243697

In [46]:
# 標準化縮放　standardized
from sklearn.preprocessing import StandardScaler

stdsc = StandardScaler()
X_train_std = stdsc.fit_transform(X_train)
X_test_std = stdsc.transform(X_test)

In [48]:
ex = pd.DataFrame([0, 1, 2, 3, 4, 5])

# standardize
ex[1] = (ex[0] - ex[0].mean()) / ex[0].std(ddof=0)

# normalize
ex[2] = (ex[0] - ex[0].min()) / (ex[0].max() - ex[0].min())
ex.columns = ['input', 'standardized', 'normalized']
ex

input  standardized  normalized
0      0      -1.46385         0.0
1      1      -0.87831         0.2
2      2      -0.29277         0.4
3      3       0.29277         0.6
4      4       0.87831         0.8
5      5       1.46385         1.0